In [43]:
# Do all necessary preprocessing, calling prepro.py
import utils
from utils import *
importlib.reload(utils)
from utils import *

In [2]:
# Load data
X, X_test_original, y = load_data() 
y = y.ravel()
scores = np.array([])
xtrain = X  
ytrain = y

In [ ]:
# # Isolation Forest in case we need it: 
# # Fit an isolation forest
# clf = IsolationForest(n_estimators=n_estimators)
# clf.fit(X_new)
# clf.decision_function(X_new)
# inlier_indices = np.where(clf.predict(X_new) == 1)[0]
# print("Size of inliers:", inlier_indices.shape)
# X_inliers = X_new[inlier_indices]
# y_inliers = y_[inlier_indices]
# X_test_inliers = X_test_new # Do not remove outliers from test dataset
# y_test_inliers = y_test
# print("Inliers shapes:", X_inliers.shape, y_inliers.shape)

# SMOTEClassifier(SMOTE(), xgb.XGBClassifier())

# Optimal Model:

In [ ]:
# Simple pipeline with grid search

steps = [("scaler", preprocessing.StandardScaler()), ("classifier", svm.SVC())]

pipeline = Pipeline(steps = steps)

parameters = {"classifier__kernel": ["rbf"],
              "classifier__gamma": ["auto"],
              "classifier__C": [0.1, 0.5, 1, 5, 10, 50, 100],
              "classifier__degree": [1, 2, 3, 4, 5, 6],
              "classifier__class_weight": ["balanced"]
             }

grid = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'balanced_accuracy')

# grid.fit(xtrain.values, ytrain.values.ravel())

# 0.7015740740740741
# {'classifier__C': 0.5, 'classifier__class_weight': 'balanced', 'classifier__degree': 1, 'classifier__gamma': 'auto', 'classifier__kernel': 'rbf'}


# Original NN model

In [ ]:
# dropout in hidden layers with weight constraint
def create_model(neurons=60):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=1000, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.5))
    model.add(Dense(neurons, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(int(neurons/2), activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.1))
    model.add(Dense(3, activation='sigmoid'))
    # Compile model
#     sgd = SGD(lr=0.02, momentum=0.9)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
 
# One hot encode data
# enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
# y_enc = enc.fit_transform(y)
y_enc = np.zeros((y.shape[0], 3))
y_enc[np.arange(y.shape[0]), y] = 1

estimators = []
estimators.append(('standardize', preprocessing.StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=1)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=3, shuffle=True)
cw = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y), y)
class_weight = {0: cw[0], 1: cw[1], 2: cw[2]}

def score(estimator, X_, y_):
    pred = estimator.predict(X_)
    y_normal = np.argmax(y_, axis=1)
    BMAC = balanced_accuracy_score(y_normal, pred)
    return BMAC


# results = cross_val_score(pipeline, X, y_enc, cv=kfold, verbose=2, scoring=score, 
#                           fit_params={'mlp__class_weight': class_weight})
# print(results)

# Keras NN Grid Search: Number of Neurons (Complete): Optimal Neurons is 30

In [3]:
inputs = [15, 30, 60, 120, 240]
results = []

for neurons in inputs:
    print("Input:", neurons)
    # dropout in hidden layers with weight constraint
    def create_model():
        # create model
        model = Sequential()
        model.add(Dense(neurons, input_dim=1000, activation='relu', kernel_constraint=maxnorm(3)))
        model.add(Dropout(0.5))
        model.add(Dense(neurons, activation='relu', kernel_constraint=maxnorm(3)))
        model.add(Dropout(0.2))
        model.add(Dense(int(neurons/2), activation='relu', kernel_constraint=maxnorm(3)))
        model.add(Dropout(0.1))
        model.add(Dense(3, activation='sigmoid'))
        # Compile model
    #     sgd = SGD(lr=0.02, momentum=0.9)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    # One hot encode data
    y_enc = np.zeros((y.shape[0], 3))
    y_enc[np.arange(y.shape[0]), y] = 1

    # Build the estimator
    estimators = []
    estimators.append(('standardize', preprocessing.StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=1)))
    pipeline = Pipeline(estimators)
    kfold = KFold(n_splits=5, shuffle=True)
    cw = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y), y)
    class_weight = {0: cw[0], 1: cw[1], 2: cw[2]}

    def score(estimator, X_, y_):
        pred = estimator.predict(X_)
        y_normal = np.argmax(y_, axis=1)
        BMAC = balanced_accuracy_score(y_normal, pred)
        return BMAC

    result = cross_val_score(pipeline, X, y_enc, cv=kfold, verbose=2, scoring=score, 
                              fit_params={'mlp__class_weight': class_weight})
    results.append(result)

# summarize results
means = [np.mean(x) for x in results]
stds = [np.std(x) for x in results]
params = inputs 
print("Best: %f using %s" % (np.max(means), params[np.argmax(means)]))
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# Save into a variable
neuron_results = results

Input: 15
[CV]  ................................................................
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Instructions for updating:
Use tf.cast instead.
Epoch 1/20
3840/3840 [==============================] - 2s 468us/step - loss: 0.6547 - acc: 0.6698
Epoch 2/20
3840/3840 [==============================] - 1s 146us/step - loss: 0.5780 - acc: 0.7444
Epoch 3/20
3840/3840 [==============================] - 1s 150us/step - loss: 0.5385 - acc: 0.7558
Epoch 4/20
3840/3840 [==============================] - 1s 200us/step - loss: 0.5133 - acc: 0.7766
Epoch 5/20
3840/3840 [==============================] - 1s 161us/step - loss: 0.5050 - acc: 0.7828
Epoch 6/20
3840/3840 [==============================] - 1s 175us/step - loss: 0.4872 - acc: 0.7955
Epoch 7/20
3840/3840 [==============================] - 1s 211us/step - loss: 0.4649 - acc: 0.8040
Epoch 8/20
3840/3840 [==============================] - 1s 162us/step - loss: 0.4726 - acc: 0.8054
Epoch 9/20
3840/3840 [==============================] - 1s 158us/step - loss: 0.4546 - acc: 0.8056
Epoch 10/20
3840/3840 [==============================] - 1s 1

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.5s remaining:    0.0s


Epoch 1/20
3840/3840 [==============================] - 2s 544us/step - loss: 0.6419 - acc: 0.6635
Epoch 2/20
3840/3840 [==============================] - 1s 194us/step - loss: 0.5651 - acc: 0.7293
Epoch 3/20
3840/3840 [==============================] - 1s 154us/step - loss: 0.5366 - acc: 0.7484
Epoch 4/20
3840/3840 [==============================] - 1s 146us/step - loss: 0.5079 - acc: 0.7688
Epoch 5/20
3840/3840 [==============================] - 1s 146us/step - loss: 0.4988 - acc: 0.7869
Epoch 6/20
3840/3840 [==============================] - 1s 179us/step - loss: 0.4907 - acc: 0.7840
Epoch 7/20
3840/3840 [==============================] - 1s 175us/step - loss: 0.4718 - acc: 0.7828
Epoch 8/20
3840/3840 [==============================] - 1s 175us/step - loss: 0.4602 - acc: 0.7836
Epoch 9/20
3840/3840 [==============================] - 1s 180us/step - loss: 0.4462 - acc: 0.7815
Epoch 10/20
3840/3840 [==============================] - 1s 191us/step - loss: 0.4320 - acc: 0.7941
Epoch 11/

...

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/20
3840/3840 [==============================] - 5s 1ms/step - loss: 0.5164 - acc: 0.7642
Epoch 2/20
3840/3840 [==============================] - 2s 533us/step - loss: 0.4446 - acc: 0.7945
Epoch 3/20
3840/3840 [==============================] - 2s 531us/step - loss: 0.4076 - acc: 0.8081
Epoch 4/20
3840/3840 [==============================] - 2s 527us/step - loss: 0.3758 - acc: 0.8264
Epoch 5/20
3840/3840 [==============================] - 2s 528us/step - loss: 0.3496 - acc: 0.8429
Epoch 6/20
3840/3840 [==============================] - 2s 535us/step - loss: 0.3396 - acc: 0.8424
Epoch 7/20
3840/3840 [==============================] - 2s 532us/step - loss: 0.3106 - acc: 0.8563
Epoch 8/20
3840/3840 [==============================] - 2s 526us/step - loss: 0.2807 - acc: 0.8678
Epoch 9/20
3840/3840 [==============================] - 2s 560us/step - loss: 0.2602 - acc: 0.8837
Epoch 10/20
3840/3840 [==============================] - 2s 550us/step - loss: 0.2398 - acc: 0.8923
Epoch 11/20

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   45.0s remaining:    0.0s


Epoch 1/20
3840/3840 [==============================] - 5s 1ms/step - loss: 0.5159 - acc: 0.7555
Epoch 2/20
3840/3840 [==============================] - 2s 530us/step - loss: 0.4463 - acc: 0.7819
Epoch 3/20
3840/3840 [==============================] - 2s 531us/step - loss: 0.4177 - acc: 0.8048
Epoch 4/20
3840/3840 [==============================] - 2s 524us/step - loss: 0.3878 - acc: 0.8177
Epoch 5/20
3840/3840 [==============================] - 2s 538us/step - loss: 0.3504 - acc: 0.8324
Epoch 6/20
3840/3840 [==============================] - 2s 540us/step - loss: 0.3237 - acc: 0.8476
Epoch 7/20
3840/3840 [==============================] - 2s 535us/step - loss: 0.3080 - acc: 0.8580
Epoch 8/20
3840/3840 [==============================] - 2s 532us/step - loss: 0.2863 - acc: 0.8683
Epoch 9/20
3840/3840 [==============================] - 2s 546us/step - loss: 0.2692 - acc: 0.8774
Epoch 10/20
3840/3840 [==============================] - 2s 530us/step - loss: 0.2444 - acc: 0.8847
Epoch 11/20

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.8min finished


In [36]:
# Analyze results

means = [np.mean(x) for x in neuron_results]
stds = [np.std(x) for x in neuron_results]
params = inputs
print("Best: %f using %s" % (np.max(means), params[np.argmax(means)]))
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# Nov 14 neuron results
# Best: 0.687105 using 30
# 0.678520 (0.015752) with: 15
# 0.687105 (0.012733) with: 30
# 0.683858 (0.024240) with: 60
# 0.670367 (0.036707) with: 120
# 0.669118 (0.013481) with: 240

# Don't need to rerun, this is good ^
for i in range(len(neuron_results)):
    plt.plot(range(0, 5, 1), neuron_results[i], label = params[i])
    plt.legend()

Best: 0.687105 using 30
0.678520 (0.015752) with: 15
0.687105 (0.012733) with: 30
0.683858 (0.024240) with: 60
0.670367 (0.036707) with: 120
0.669118 (0.013481) with: 240


# Keras NN Search: Dropout and Max Norm (Complete) Best: (0.7, 4)

In [42]:
weight_c = [1, 2, 3, 4, 5]
dropout = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
results = []

for dropout_rate, weight_constraint in itertools.product(dropout, weight_c):
    print("Input:", (dropout_rate, weight_constraint))
    # dropout in hidden layers with weight constraint
    def create_model():
        # create model
        neurons = 30
        model = Sequential()
        model.add(Dense(neurons, input_dim=1000, activation='relu', kernel_constraint=maxnorm(weight_constraint)))
        model.add(Dropout(dropout_rate))
        model.add(Dense(neurons, activation='relu', kernel_constraint=maxnorm(weight_constraint)))
        model.add(Dropout(dropout_rate*0.4))
        model.add(Dense(int(neurons/2), activation='relu', kernel_constraint=maxnorm(weight_constraint)))
        model.add(Dropout(dropout_rate*0.2))
        model.add(Dense(3, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    # One hot encode data
    y_enc = np.zeros((y.shape[0], 3))
    y_enc[np.arange(y.shape[0]), y] = 1

    # Build the estimator
    estimators = []
    estimators.append(('standardize', preprocessing.StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=1)))
    pipeline = Pipeline(estimators)
    kfold = KFold(n_splits=5, shuffle=True)
    cw = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y), y)
    class_weight = {0: cw[0], 1: cw[1], 2: cw[2]}

    def score(estimator, X_, y_):
        pred = estimator.predict(X_)
        y_normal = np.argmax(y_, axis=1)
        BMAC = balanced_accuracy_score(y_normal, pred)
        return BMAC

    result = cross_val_score(pipeline, X, y_enc, cv=kfold, verbose=3, scoring=score, 
                              fit_params={'mlp__class_weight': class_weight})
    results.append(result)

# summarize results
means = [np.mean(x) for x in results]
stds = [np.std(x) for x in results]
params = list(itertools.product(dropout, weight_c))
print("Best: %f using %s" % (np.max(means), params[np.argmax(means)]))
for m, st, p in zip(means, stds, params):
    print("%f (%f) with: %r" % (m, st, p))
    
# Save into a variable
dropout_results = results

Input: (0.0, 1)
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/20
3840/3840 [==============================] - 8s 2ms/step - loss: 0.5033 - acc: 0.7655
Epoch 2/20
3840/3840 [==============================] - 1s 297us/step - loss: 0.4104 - acc: 0.8090
Epoch 3/20
3840/3840 [==============================] - 1s 256us/step - loss: 0.3543 - acc: 0.8386
Epoch 4/20
3840/3840 [==============================] - 1s 248us/step - loss: 0.3087 - acc: 0.8640
Epoch 5/20
3840/3840 [==============================] - 1s 225us/step - loss: 0.2865 - acc: 0.8762
Epoch 6/20
3840/3840 [==============================] - 1s 258us/step - loss: 0.2412 - acc: 0.8959
Epoch 7/20
3840/3840 [==============================] - 1s 220us/step - loss: 0.2194 - acc: 0.9147
Epoch 8/20
3840/3840 [==============================] - 1s 263us/step - loss: 0.1950 - acc: 0.9214
Epoch 9/20
3840/3840 [==============================] - 1s 250us/step - loss: 0.1971 - acc: 0.9240
Epoch 10/20
3840/3840 [==============================] - 1s 268us/step - loss: 0.1730 - acc: 0.9288
Epoch 11/20

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.5s remaining:    0.0s


Epoch 1/20
3840/3840 [==============================] - 8s 2ms/step - loss: 0.4961 - acc: 0.7398
Epoch 2/20
3840/3840 [==============================] - 1s 270us/step - loss: 0.3981 - acc: 0.8074
Epoch 3/20
3840/3840 [==============================] - 1s 228us/step - loss: 0.3554 - acc: 0.8313
Epoch 4/20
3840/3840 [==============================] - 1s 264us/step - loss: 0.3054 - acc: 0.8528
Epoch 5/20
3840/3840 [==============================] - 1s 242us/step - loss: 0.2835 - acc: 0.8649
Epoch 6/20
3840/3840 [==============================] - 1s 274us/step - loss: 0.2531 - acc: 0.8813
Epoch 7/20
3840/3840 [==============================] - 1s 235us/step - loss: 0.2205 - acc: 0.9011
Epoch 8/20
3840/3840 [==============================] - 1s 261us/step - loss: 0.1918 - acc: 0.9118
Epoch 9/20
3840/3840 [==============================] - 1s 231us/step - loss: 0.1710 - acc: 0.9205
Epoch 10/20
3840/3840 [==============================] - 1s 271us/step - loss: 0.1419 - acc: 0.9363
Epoch 11/20

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.6min remaining:    0.0s


Epoch 1/20
3840/3840 [==============================] - 36s 9ms/step - loss: 0.7483 - acc: 0.6707
Epoch 2/20
3840/3840 [==============================] - 2s 607us/step - loss: 0.6389 - acc: 0.7516
Epoch 3/20
3840/3840 [==============================] - 2s 605us/step - loss: 0.6211 - acc: 0.7660
Epoch 4/20
3840/3840 [==============================] - 2s 620us/step - loss: 0.5778 - acc: 0.7772
Epoch 5/20
3840/3840 [==============================] - 2s 610us/step - loss: 0.5832 - acc: 0.7796
Epoch 6/20
3840/3840 [==============================] - 2s 606us/step - loss: 0.5650 - acc: 0.7846
Epoch 7/20
3840/3840 [==============================] - 2s 605us/step - loss: 0.5474 - acc: 0.7896
Epoch 8/20
3840/3840 [==============================] - 2s 604us/step - loss: 0.5552 - acc: 0.7857
Epoch 9/20
3840/3840 [==============================] - 2s 606us/step - loss: 0.5474 - acc: 0.7900
Epoch 10/20
3840/3840 [==============================] - 2s 608us/step - loss: 0.5476 - acc: 0.7906
Epoch 11/2

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.2min remaining:    0.0s


Epoch 1/20
3840/3840 [==============================] - 37s 10ms/step - loss: 0.9650 - acc: 0.5471
Epoch 2/20
3840/3840 [==============================] - 2s 623us/step - loss: 0.7086 - acc: 0.6706
Epoch 3/20
3840/3840 [==============================] - 2s 614us/step - loss: 0.6579 - acc: 0.7198
Epoch 4/20
3840/3840 [==============================] - 2s 619us/step - loss: 0.6071 - acc: 0.7469
Epoch 5/20
3840/3840 [==============================] - 2s 616us/step - loss: 0.5896 - acc: 0.7671
Epoch 6/20
3840/3840 [==============================] - 2s 609us/step - loss: 0.5719 - acc: 0.7673
Epoch 7/20
3840/3840 [==============================] - 2s 612us/step - loss: 0.5607 - acc: 0.7762
Epoch 8/20
3840/3840 [==============================] - 2s 613us/step - loss: 0.5548 - acc: 0.7768
Epoch 9/20
3840/3840 [==============================] - 2s 606us/step - loss: 0.5461 - acc: 0.7825
Epoch 10/20
3840/3840 [==============================] - 2s 637us/step - loss: 0.5503 - acc: 0.7824
Epoch 11/

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  8.1min finished


In [44]:
# Analyze Results

means = [np.mean(x) for x in dropout_results]
stds = [np.std(x) for x in dropout_results]
params = list(itertools.product(dropout, weight_c))
print("Best: %f using %s" % (np.max(means), params[np.argmax(means)]))
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Nov 14 dropout results (60 neurons)
# Best: 0.678588 using (0.4, 5)
# 0.640552 (0.027379) with: (0.0, 1)
# 0.649154 (0.009938) with: (0.1, 2)
# 0.643685 (0.016983) with: (0.2, 3)
# 0.668004 (0.019197) with: (0.3, 4)
# 0.678588 (0.025227) with: (0.4, 5)

# Nov 15 Dropout
# Best: 0.696174 using (0.7, 4)
# 0.650642 (0.015216) with: (0.0, 1)
# 0.657307 (0.026931) with: (0.0, 2)
# 0.629835 (0.029415) with: (0.0, 3)
# 0.653656 (0.027232) with: (0.0, 4)
# 0.636517 (0.013191) with: (0.0, 5)
# 0.664681 (0.022337) with: (0.1, 1)
# 0.654195 (0.021384) with: (0.1, 2)
# 0.656906 (0.016353) with: (0.1, 3)
# 0.661421 (0.025813) with: (0.1, 4)
# 0.653796 (0.025165) with: (0.1, 5)
# 0.669235 (0.020462) with: (0.2, 1)
# 0.671460 (0.019962) with: (0.2, 2)
# 0.659945 (0.016278) with: (0.2, 3)
# 0.663501 (0.015111) with: (0.2, 4)
# 0.652485 (0.014111) with: (0.2, 5)
# 0.664239 (0.021049) with: (0.3, 1)
# 0.681309 (0.012280) with: (0.3, 2)
# 0.666394 (0.035302) with: (0.3, 3)
# 0.662096 (0.018217) with: (0.3, 4)
# 0.669694 (0.011330) with: (0.3, 5)
# 0.669241 (0.018018) with: (0.4, 1)
# 0.674562 (0.019739) with: (0.4, 2)
# 0.677877 (0.017999) with: (0.4, 3)
# 0.678872 (0.021842) with: (0.4, 4)
# 0.673720 (0.030201) with: (0.4, 5)
# 0.689792 (0.025039) with: (0.5, 1)
# 0.686033 (0.021267) with: (0.5, 2)
# 0.684664 (0.017476) with: (0.5, 3)
# 0.676223 (0.027741) with: (0.5, 4)
# 0.671119 (0.023597) with: (0.5, 5)
# 0.690366 (0.006684) with: (0.6, 1)
# 0.683671 (0.029502) with: (0.6, 2)
# 0.691728 (0.024163) with: (0.6, 3)
# 0.679865 (0.010925) with: (0.6, 4)
# 0.682480 (0.021617) with: (0.6, 5)
# 0.682142 (0.005927) with: (0.7, 1)
# 0.686899 (0.019932) with: (0.7, 2)
# 0.690437 (0.028829) with: (0.7, 3)
# 0.696174 (0.014894) with: (0.7, 4)
# 0.688140 (0.007276) with: (0.7, 5)
# 0.683688 (0.015699) with: (0.8, 1)
# 0.656767 (0.023636) with: (0.8, 2)
# 0.668052 (0.022368) with: (0.8, 3)
# 0.668679 (0.015473) with: (0.8, 4)
# 0.652830 (0.025408) with: (0.8, 5)
# 0.575628 (0.041368) with: (0.9, 1)
# 0.551756 (0.023067) with: (0.9, 2)
# 0.555779 (0.013684) with: (0.9, 3)
# 0.544907 (0.026359) with: (0.9, 4)
# 0.563261 (0.035773) with: (0.9, 5)

Best: 0.696174 using (0.7, 4)
0.650642 (0.015216) with: (0.0, 1)
0.657307 (0.026931) with: (0.0, 2)
0.629835 (0.029415) with: (0.0, 3)
0.653656 (0.027232) with: (0.0, 4)
0.636517 (0.013191) with: (0.0, 5)
0.664681 (0.022337) with: (0.1, 1)
0.654195 (0.021384) with: (0.1, 2)
0.656906 (0.016353) with: (0.1, 3)
0.661421 (0.025813) with: (0.1, 4)
0.653796 (0.025165) with: (0.1, 5)
0.669235 (0.020462) with: (0.2, 1)
0.671460 (0.019962) with: (0.2, 2)
0.659945 (0.016278) with: (0.2, 3)
0.663501 (0.015111) with: (0.2, 4)
0.652485 (0.014111) with: (0.2, 5)
0.664239 (0.021049) with: (0.3, 1)
0.681309 (0.012280) with: (0.3, 2)
0.666394 (0.035302) with: (0.3, 3)
0.662096 (0.018217) with: (0.3, 4)
0.669694 (0.011330) with: (0.3, 5)
0.669241 (0.018018) with: (0.4, 1)
0.674562 (0.019739) with: (0.4, 2)
0.677877 (0.017999) with: (0.4, 3)
0.678872 (0.021842) with: (0.4, 4)
0.673720 (0.030201) with: (0.4, 5)
0.689792 (0.025039) with: (0.5, 1)
0.686033 (0.021267) with: (0.5, 2)
0.684664 (0.017476) with:

# Keras NN Search: Batch Size and Number of Epochs (Complete) Best: (80, 20)

In [3]:
batch_size_l = [10, 20, 40, 60, 80, 100]
epochs_l = [10, 20, 50, 100]
results = []

for batch_size, epochs in itertools.product(batch_size_l, epochs_l):
    print("Input:", (batch_size, epochs))
    # dropout in hidden layers with weight constraint
    def create_model():
        # create model
        neurons = 30 # From previous optimization
        dropout_rate = 0.7 # From previous optimization
        weight_constraint = 4 # From previous optimization
        model = Sequential()
        model.add(Dense(neurons, input_dim=1000, activation='relu', kernel_constraint=maxnorm(weight_constraint)))
        model.add(Dropout(dropout_rate))
        model.add(Dense(neurons, activation='relu', kernel_constraint=maxnorm(weight_constraint)))
        model.add(Dropout(dropout_rate*0.4))
        model.add(Dense(int(neurons/2), activation='relu', kernel_constraint=maxnorm(weight_constraint)))
        model.add(Dropout(dropout_rate*0.2))
        model.add(Dense(3, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    # One hot encode data
    y_enc = np.zeros((y.shape[0], 3))
    y_enc[np.arange(y.shape[0]), y] = 1

    # Build the estimator
    estimators = []
    estimators.append(('standardize', preprocessing.StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=1)))
    pipeline = Pipeline(estimators)
    kfold = KFold(n_splits=5, shuffle=True)
    cw = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y), y)
    class_weight = {0: cw[0], 1: cw[1], 2: cw[2]}

    def score(estimator, X_, y_):
        pred = estimator.predict(X_)
        y_normal = np.argmax(y_, axis=1)
        BMAC = balanced_accuracy_score(y_normal, pred)
        return BMAC

    result = cross_val_score(pipeline, X, y_enc, cv=kfold, verbose=3, scoring=score, 
                              fit_params={'mlp__class_weight': class_weight})
    results.append(result)

# summarize results
means = [np.mean(x) for x in results]
stds = [np.std(x) for x in results]
params = list(itertools.product(batch_size_l, epochs_l))
print("Best: %f using %s" % (np.max(means), params[np.argmax(means)]))
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# Save into a variable
batchepoch_results = results

Input: (10, 10)
[CV]  ................................................................
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Instructions for updating:
Use tf.cast instead.
Epoch 1/10
3840/3840 [==============================] - 3s 875us/step - loss: 0.6479 - acc: 0.6914
Epoch 2/10
3840/3840 [==============================] - 2s 535us/step - loss: 0.5581 - acc: 0.7642
Epoch 3/10
3840/3840 [==============================] - 2s 558us/step - loss: 0.5232 - acc: 0.7871
Epoch 4/10
3840/3840 [==============================] - 2s 534us/step - loss: 0.5108 - acc: 0.7922
Epoch 5/10
3840/3840 [==============================] - 2s 544us/step - loss: 0.5037 - acc: 0.7943
Epoch 6/10
3840/3840 [==============================] - 2s 545us/step - loss: 0.4922 - acc: 0.7946
Epoch 7/10
3840/3840 [==============================] - 2s 555us/step - loss: 0.4841 - acc: 0.7913
Epoch 8/10
3840/3840 [==============================] - 2s 552us/step - loss: 0.4613 - acc: 0.7879
Epoch 9/10
3840/3840 [==============================] - 2s 548us/step - loss: 0.4623 - acc: 0.7898
Epoch 10/10
960/960 [==============================] - 0s 247

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.5s remaining:    0.0s


Epoch 1/10
3840/3840 [==============================] - 4s 925us/step - loss: 0.6449 - acc: 0.6762
Epoch 2/10
3840/3840 [==============================] - 2s 551us/step - loss: 0.5452 - acc: 0.7541
Epoch 3/10
3840/3840 [==============================] - 2s 578us/step - loss: 0.5158 - acc: 0.7600
Epoch 4/10
3840/3840 [==============================] - 2s 559us/step - loss: 0.5003 - acc: 0.7642
Epoch 5/10
3840/3840 [==============================] - 2s 556us/step - loss: 0.4994 - acc: 0.7737
Epoch 6/10
3840/3840 [==============================] - 2s 575us/step - loss: 0.4776 - acc: 0.7843
Epoch 7/10
3840/3840 [==============================] - 2s 570us/step - loss: 0.4780 - acc: 0.7810
Epoch 8/10
3840/3840 [==============================] - 2s 560us/step - loss: 0.4551 - acc: 0.7832
Epoch 9/10
3840/3840 [==============================] - 2s 559us/step - loss: 0.4586 - acc: 0.7917
Epoch 10/10
960/960 [==============================] - 0s 466us/step
[CV] ...................................

...

Epoch 1/100
3840/3840 [==============================] - 12s 3ms/step - loss: 0.6940 - acc: 0.6397
Epoch 2/100
3840/3840 [==============================] - 0s 114us/step - loss: 0.6243 - acc: 0.7257
Epoch 3/100
3840/3840 [==============================] - 1s 140us/step - loss: 0.5881 - acc: 0.7458
Epoch 4/100
3840/3840 [==============================] - 0s 114us/step - loss: 0.5591 - acc: 0.7508
Epoch 5/100
3840/3840 [==============================] - 0s 116us/step - loss: 0.5554 - acc: 0.7553
Epoch 6/100
3840/3840 [==============================] - 0s 115us/step - loss: 0.5222 - acc: 0.7760
Epoch 7/100
3840/3840 [==============================] - 0s 122us/step - loss: 0.5129 - acc: 0.7811
Epoch 8/100
3840/3840 [==============================] - 0s 130us/step - loss: 0.5106 - acc: 0.7722
Epoch 9/100
3840/3840 [==============================] - 0s 116us/step - loss: 0.5002 - acc: 0.7801
Epoch 10/100
3840/3840 [==============================] - 0s 113us/step - loss: 0.4886 - acc: 0.7902


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s


Epoch 1/100
3840/3840 [==============================] - 12s 3ms/step - loss: 0.7033 - acc: 0.5576
Epoch 2/100
3840/3840 [==============================] - 0s 115us/step - loss: 0.6067 - acc: 0.7030
Epoch 3/100
3840/3840 [==============================] - 0s 113us/step - loss: 0.5640 - acc: 0.7309
Epoch 4/100
3840/3840 [==============================] - 0s 123us/step - loss: 0.5534 - acc: 0.7371
Epoch 5/100
3840/3840 [==============================] - 0s 114us/step - loss: 0.5294 - acc: 0.7555
Epoch 6/100
3840/3840 [==============================] - 0s 115us/step - loss: 0.5337 - acc: 0.7588
Epoch 7/100
3840/3840 [==============================] - 0s 114us/step - loss: 0.5186 - acc: 0.7679
Epoch 8/100
3840/3840 [==============================] - 0s 122us/step - loss: 0.5001 - acc: 0.7727
Epoch 9/100
3840/3840 [==============================] - 0s 118us/step - loss: 0.4969 - acc: 0.7714
Epoch 10/100
3840/3840 [==============================] - 0s 111us/step - loss: 0.4841 - acc: 0.7806


3840/3840 [==============================] - 0s 128us/step - loss: 0.2991 - acc: 0.8760
Epoch 62/100
3840/3840 [==============================] - 0s 117us/step - loss: 0.2847 - acc: 0.8701
Epoch 63/100
3840/3840 [==============================] - 0s 114us/step - loss: 0.2896 - acc: 0.8795
Epoch 64/100
3840/3840 [==============================] - 0s 115us/step - loss: 0.2728 - acc: 0.8762
Epoch 65/100
3840/3840 [==============================] - 0s 116us/step - loss: 0.2904 - acc: 0.8774
Epoch 66/100
3840/3840 [==============================] - 0s 124us/step - loss: 0.3006 - acc: 0.8703
Epoch 67/100
3840/3840 [==============================] - 0s 113us/step - loss: 0.2883 - acc: 0.8825
Epoch 68/100
3840/3840 [==============================] - 0s 116us/step - loss: 0.3098 - acc: 0.8724
Epoch 69/100
3840/3840 [==============================] - 0s 114us/step - loss: 0.2965 - acc: 0.8739
Epoch 70/100
3840/3840 [==============================] - 1s 134us/step - loss: 0.2866 - acc: 0.8755
Epo

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  5.2min finished


In [4]:
# Analyze Results

means = [np.mean(x) for x in batchepoch_results]
stds = [np.std(x) for x in batchepoch_results]
params = list(itertools.product(batch_size_l, epochs_l))
print("Best: %f using %s" % (np.max(means), params[np.argmax(means)]))
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Nov 14 Bach Epoch Results (60 neurons)
# Best: 0.671280 using (10, 10)
# 0.671280 (0.025807) with: (10, 10)
# 0.664303 (0.015683) with: (20, 50)
# 0.638915 (0.030326) with: (40, 100)

# Nov 15 opt (30 neurons)
# Best: 0.686079 using (80, 20)
# 0.666221 (0.019225) with: (10, 10)
# 0.682927 (0.017058) with: (10, 20)
# 0.684848 (0.012817) with: (10, 50)
# 0.663020 (0.018521) with: (10, 100)
# 0.670920 (0.017642) with: (20, 10)
# 0.681969 (0.041114) with: (20, 20)
# 0.672319 (0.022787) with: (20, 50)
# 0.668099 (0.008249) with: (20, 100)
# 0.671866 (0.013040) with: (40, 10)
# 0.681963 (0.016211) with: (40, 20)
# 0.682536 (0.026968) with: (40, 50)
# 0.661211 (0.019936) with: (40, 100)
# 0.671109 (0.019507) with: (60, 10)
# 0.682207 (0.016815) with: (60, 20)
# 0.670250 (0.016813) with: (60, 50)
# 0.675514 (0.020505) with: (60, 100)
# 0.671250 (0.044379) with: (80, 10)
# 0.686079 (0.012543) with: (80, 20)
# 0.679829 (0.018463) with: (80, 50)
# 0.678590 (0.012564) with: (80, 100)
# 0.661263 (0.015685) with: (100, 10)
# 0.682316 (0.020151) with: (100, 20)
# 0.679139 (0.017597) with: (100, 50)
# 0.660951 (0.017247) with: (100, 100)

Best: 0.686079 using (80, 20)
0.666221 (0.019225) with: (10, 10)
0.682927 (0.017058) with: (10, 20)
0.684848 (0.012817) with: (10, 50)
0.663020 (0.018521) with: (10, 100)
0.670920 (0.017642) with: (20, 10)
0.681969 (0.041114) with: (20, 20)
0.672319 (0.022787) with: (20, 50)
0.668099 (0.008249) with: (20, 100)
0.671866 (0.013040) with: (40, 10)
0.681963 (0.016211) with: (40, 20)
0.682536 (0.026968) with: (40, 50)
0.661211 (0.019936) with: (40, 100)
0.671109 (0.019507) with: (60, 10)
0.682207 (0.016815) with: (60, 20)
0.670250 (0.016813) with: (60, 50)
0.675514 (0.020505) with: (60, 100)
0.671250 (0.044379) with: (80, 10)
0.686079 (0.012543) with: (80, 20)
0.679829 (0.018463) with: (80, 50)
0.678590 (0.012564) with: (80, 100)
0.661263 (0.015685) with: (100, 10)
0.682316 (0.020151) with: (100, 20)
0.679139 (0.017597) with: (100, 50)
0.660951 (0.017247) with: (100, 100)


# Submit best NN Model so far

In [22]:
# dropout in hidden layers with weight constraint
def create_model():
    # create model
    neurons = 30
    dropout_rate = 0.7
    weight_constraint = 4
    model = Sequential()
    model.add(Dense(neurons, input_dim=1000, activation='relu', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, activation='relu', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate*0.4))
    model.add(Dense(int(neurons/2), activation='relu', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate*0.2))
    model.add(Dense(3, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# One hot encode data
y_enc = np.zeros((y.shape[0], 3))
y_enc[np.arange(y.shape[0]), y] = 1

# Build the estimator
estimators = []
estimators.append(('standardize', preprocessing.StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=25, batch_size=35, verbose=1)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5, shuffle=True)
cw = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y), y)
class_weight = {0: cw[0], 1: cw[1], 2: cw[2]}

def score(estimator, X_, y_):
    pred = estimator.predict(X_)
    y_normal = np.argmax(y_, axis=1)
    BMAC = balanced_accuracy_score(y_normal, pred)
    return BMAC

result = cross_val_score(pipeline, X, y_enc, cv=kfold, verbose=3, scoring=score, 
                          fit_params={'mlp__class_weight': class_weight})

# NN2 cv:


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
Epoch 1/25
3840/3840 [==============================] - 38s 10ms/step - loss: 0.6566 - acc: 0.6742
Epoch 2/25
3840/3840 [==============================] - 2s 565us/step - loss: 0.5747 - acc: 0.7575
Epoch 3/25
3840/3840 [==============================] - 2s 575us/step - loss: 0.5659 - acc: 0.7619
Epoch 4/25
3840/3840 [==============================] - 2s 572us/step - loss: 0.5300 - acc: 0.7746
Epoch 5/25
3840/3840 [==============================] - 2s 573us/step - loss: 0.5198 - acc: 0.7807
Epoch 6/25
3840/3840 [==============================] - 2s 571us/step - loss: 0.5120 - acc: 0.7802
Epoch 7/25
3840/3840 [==============================] - 2s 596us/step - loss: 0.5048 - acc: 0.7772
Epoch 8/25
3840/3840 [==============================] - 2s 571us/step - loss: 0.4790 - acc: 0.7879
Epoch 9/25
3840/3840 [==============================] - 2s 577us/step - loss: 0.4737 - acc: 0.7905
Epoch 10/25
3840/3840 [===============

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.8min remaining:    0.0s


Epoch 1/25
3840/3840 [==============================] - 37s 10ms/step - loss: 0.6687 - acc: 0.6046
Epoch 2/25
3840/3840 [==============================] - 2s 560us/step - loss: 0.5738 - acc: 0.7214
Epoch 3/25
3840/3840 [==============================] - 2s 561us/step - loss: 0.5435 - acc: 0.7520
Epoch 4/25
3840/3840 [==============================] - 2s 565us/step - loss: 0.5230 - acc: 0.7652
Epoch 5/25
3840/3840 [==============================] - 2s 558us/step - loss: 0.4925 - acc: 0.7772
Epoch 6/25
3840/3840 [==============================] - 2s 560us/step - loss: 0.4842 - acc: 0.7831
Epoch 7/25
3840/3840 [==============================] - 2s 579us/step - loss: 0.4684 - acc: 0.7918
Epoch 8/25
3840/3840 [==============================] - 2s 568us/step - loss: 0.4639 - acc: 0.7892
Epoch 9/25
3840/3840 [==============================] - 2s 565us/step - loss: 0.4464 - acc: 0.7975
Epoch 10/25
3840/3840 [==============================] - 2s 568us/step - loss: 0.4385 - acc: 0.7984
Epoch 11/

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.6min remaining:    0.0s


Epoch 1/25
3840/3840 [==============================] - 40s 10ms/step - loss: 0.6563 - acc: 0.7807
Epoch 2/25
3840/3840 [==============================] - 2s 554us/step - loss: 0.5804 - acc: 0.8039
Epoch 3/25
3840/3840 [==============================] - 2s 547us/step - loss: 0.5650 - acc: 0.8001
Epoch 4/25
3840/3840 [==============================] - 2s 554us/step - loss: 0.5380 - acc: 0.8023
Epoch 5/25
3840/3840 [==============================] - 2s 555us/step - loss: 0.5330 - acc: 0.8029
Epoch 6/25
3840/3840 [==============================] - 2s 559us/step - loss: 0.5183 - acc: 0.7953
Epoch 7/25
3840/3840 [==============================] - 2s 554us/step - loss: 0.5005 - acc: 0.8038
Epoch 8/25
3840/3840 [==============================] - 2s 553us/step - loss: 0.4928 - acc: 0.7959
Epoch 9/25
3840/3840 [==============================] - 2s 563us/step - loss: 0.4849 - acc: 0.8006
Epoch 10/25
3840/3840 [==============================] - 2s 549us/step - loss: 0.4824 - acc: 0.7904
Epoch 11/

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  9.2min finished


In [24]:
# Scale data
print(np.mean(result))
xtest = X_test_original
scaler = preprocessing.StandardScaler()
xtest_scaled = scaler.fit_transform(xtest)
scaler = preprocessing.StandardScaler()
xtrain_scaled = scaler.fit_transform(xtrain)

# One hot encode data
y_enc = np.zeros((y.shape[0], 3))
y_enc[np.arange(y.shape[0]), y] = 1

# Fit the model
cw = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y), y)
class_weight = {0: cw[0], 1: cw[1], 2: cw[2]}
pipeline = Pipeline(estimators)
pipeline.fit(xtrain_scaled, y_enc, mlp__class_weight = class_weight)
pred = pipeline.predict(xtest_scaled)

index = pd.read_csv("sample.csv")
index['y'] = pred

index.to_csv("predictionsSimpleGridCVNN2.csv")

0.6837588203178561
Epoch 1/25
4800/4800 [==============================] - 44s 9ms/step - loss: 0.6789 - acc: 0.6094
Epoch 2/25
4800/4800 [==============================] - 3s 682us/step - loss: 0.5772 - acc: 0.7394
Epoch 3/25
4800/4800 [==============================] - 3s 709us/step - loss: 0.5388 - acc: 0.7669
Epoch 4/25
4800/4800 [==============================] - 3s 680us/step - loss: 0.5299 - acc: 0.7719
Epoch 5/25
4800/4800 [==============================] - 3s 628us/step - loss: 0.5147 - acc: 0.7792
Epoch 6/25
4800/4800 [==============================] - 3s 603us/step - loss: 0.4907 - acc: 0.7890
Epoch 7/25
4800/4800 [==============================] - 3s 605us/step - loss: 0.4771 - acc: 0.7869
Epoch 8/25
4800/4800 [==============================] - 3s 608us/step - loss: 0.4660 - acc: 0.7958
Epoch 9/25
4800/4800 [==============================] - 3s 654us/step - loss: 0.4545 - acc: 0.7926
Epoch 10/25
4800/4800 [==============================] - 3s 652us/step - loss: 0.4461 - acc

# Run final grid search on NN: Batch size. Complete. Best: 0.695380 using (35, 25). Call this model NN2. 

In [20]:
batch_size_l = [25, 32, 35, 40, 50, 60, 70, 80, 90]
epochs = [17, 20, 25, 30]
results = []

for batch_size, epoch in itertools.product(batch_size_l, epochs):
    print("Input:", (batch_size, epoch))
    # dropout in hidden layers with weight constraint
    def create_model():
        # create model
        neurons = 30 # From previous optimization
        dropout_rate = 0.7 # From previous optimization
        weight_constraint = 4 # From previous optimization
        model = Sequential()
        model.add(Dense(neurons, input_dim=1000, activation='relu', kernel_constraint=maxnorm(weight_constraint)))
        model.add(Dropout(dropout_rate))
        model.add(Dense(neurons, activation='relu', kernel_constraint=maxnorm(weight_constraint)))
        model.add(Dropout(dropout_rate*0.4))
        model.add(Dense(int(neurons/2), activation='relu', kernel_constraint=maxnorm(weight_constraint)))
        model.add(Dropout(dropout_rate*0.2))
        model.add(Dense(3, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    # One hot encode data
    y_enc = np.zeros((y.shape[0], 3))
    y_enc[np.arange(y.shape[0]), y] = 1

    # Build the estimator
    estimators = []
    estimators.append(('standardize', preprocessing.StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=epoch, batch_size=batch_size, verbose=1)))
    pipeline = Pipeline(estimators)
    kfold = KFold(n_splits=5, shuffle=True)
    cw = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y), y)
    class_weight = {0: cw[0], 1: cw[1], 2: cw[2]}

    def score(estimator, X_, y_):
        pred = estimator.predict(X_)
        y_normal = np.argmax(y_, axis=1)
        BMAC = balanced_accuracy_score(y_normal, pred)
        return BMAC

    result = cross_val_score(pipeline, X, y_enc, cv=kfold, verbose=3, scoring=score, 
                              fit_params={'mlp__class_weight': class_weight})
    results.append(result)

# summarize results
means = [np.mean(x) for x in results]
stds = [np.std(x) for x in results]
params = list(itertools.product(batch_size_l, epochs))
print("Best: %f using %s" % (np.max(means), params[np.argmax(means)]))
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# Save into a variable
batchepoch2_results = results

# Results:
# Best: 0.695380 using (35, 25)
# 0.692537 (0.022636) with: (25, 17)
# 0.685639 (0.018829) with: (25, 20)
# 0.682780 (0.025608) with: (25, 25)
# 0.684346 (0.017185) with: (25, 30)
# 0.691154 (0.014746) with: (32, 17)
# 0.688317 (0.008964) with: (32, 20)
# 0.678900 (0.006081) with: (32, 25)
# 0.689969 (0.019698) with: (32, 30)
# 0.673499 (0.013015) with: (35, 17)
# 0.685111 (0.019830) with: (35, 20)
# 0.695380 (0.011148) with: (35, 25)
# 0.681501 (0.022806) with: (35, 30)
# 0.674983 (0.012844) with: (40, 17)
# 0.682939 (0.014346) with: (40, 20)
# 0.683199 (0.012419) with: (40, 25)
# 0.686257 (0.026148) with: (40, 30)
# 0.688066 (0.006543) with: (50, 17)
# 0.683393 (0.011736) with: (50, 20)
# 0.679720 (0.006632) with: (50, 25)
# 0.683576 (0.021029) with: (50, 30)
# 0.673349 (0.011632) with: (60, 17)
# 0.681240 (0.025178) with: (60, 20)
# 0.682025 (0.024088) with: (60, 25)
# 0.683618 (0.008155) with: (60, 30)
# 0.671850 (0.019080) with: (70, 17)
# 0.684717 (0.025766) with: (70, 20)
# 0.687561 (0.023567) with: (70, 25)
# 0.687506 (0.017302) with: (70, 30)
# 0.676964 (0.013289) with: (80, 17)
# 0.676168 (0.016125) with: (80, 20)
# 0.684666 (0.025443) with: (80, 25)
# 0.686619 (0.010823) with: (80, 30)
# 0.666451 (0.020302) with: (90, 17)
# 0.679666 (0.015284) with: (90, 20)
# 0.680232 (0.026340) with: (90, 25)
# 0.673651 (0.027212) with: (90, 30)

Input: (25, 17)
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/17
3840/3840 [==============================] - 15s 4ms/step - loss: 0.6486 - acc: 0.7072
Epoch 2/17
3840/3840 [==============================] - 2s 464us/step - loss: 0.5847 - acc: 0.7651
Epoch 3/17
3840/3840 [==============================] - 2s 441us/step - loss: 0.5538 - acc: 0.7701
Epoch 4/17
3840/3840 [==============================] - 2s 419us/step - loss: 0.5330 - acc: 0.7678
Epoch 5/17
3840/3840 [==============================] - 2s 432us/step - loss: 0.5160 - acc: 0.7688
Epoch 6/17
3840/3840 [==============================] - 2s 424us/step - loss: 0.5107 - acc: 0.7666
Epoch 7/17
3840/3840 [==============================] - 2s 506us/step - loss: 0.4864 - acc: 0.7668
Epoch 8/17
3840/3840 [==============================] - 2s 480us/step - loss: 0.4803 - acc: 0.7691
Epoch 9/17
3840/3840 [==============================] - 3s 661us/step - loss: 0.4828 - acc: 0.7681
Epoch 10/17
3840/3840 [==============================] - 3s 794us/step - loss: 0.4695 - acc: 0.7671
Epoch 11/1

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   53.7s remaining:    0.0s


Epoch 1/17
3840/3840 [==============================] - 15s 4ms/step - loss: 0.6772 - acc: 0.6168
Epoch 2/17
3840/3840 [==============================] - 2s 410us/step - loss: 0.5606 - acc: 0.7539
Epoch 3/17
3840/3840 [==============================] - 2s 426us/step - loss: 0.5405 - acc: 0.7656
Epoch 4/17
3840/3840 [==============================] - 2s 412us/step - loss: 0.5128 - acc: 0.7832
Epoch 5/17
3840/3840 [==============================] - 2s 432us/step - loss: 0.5035 - acc: 0.7885
Epoch 6/17
3840/3840 [==============================] - 2s 466us/step - loss: 0.4915 - acc: 0.7910
Epoch 7/17
3840/3840 [==============================] - 2s 418us/step - loss: 0.4777 - acc: 0.7940
Epoch 8/17
3840/3840 [==============================] - 2s 635us/step - loss: 0.4659 - acc: 0.8030
Epoch 9/17
3840/3840 [==============================] - 2s 578us/step - loss: 0.4660 - acc: 0.8034
Epoch 10/17
3840/3840 [==============================] - 2s 488us/step - loss: 0.4614 - acc: 0.8070
Epoch 11/1

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s


Epoch 1/30
3840/3840 [==============================] - 34s 9ms/step - loss: 0.7770 - acc: 0.4497
Epoch 2/30
3840/3840 [==============================] - 1s 218us/step - loss: 0.6559 - acc: 0.6171
Epoch 3/30
3840/3840 [==============================] - 1s 218us/step - loss: 0.6137 - acc: 0.6822
Epoch 4/30
3840/3840 [==============================] - 1s 235us/step - loss: 0.5844 - acc: 0.7265
Epoch 5/30
3840/3840 [==============================] - 1s 220us/step - loss: 0.5514 - acc: 0.7391
Epoch 6/30
3840/3840 [==============================] - 1s 239us/step - loss: 0.5338 - acc: 0.7609
Epoch 7/30
3840/3840 [==============================] - 1s 219us/step - loss: 0.5288 - acc: 0.7556
Epoch 8/30
3840/3840 [==============================] - 1s 216us/step - loss: 0.5173 - acc: 0.7591
Epoch 9/30
3840/3840 [==============================] - 1s 240us/step - loss: 0.4984 - acc: 0.7707
Epoch 10/30
3840/3840 [==============================] - 1s 219us/step - loss: 0.4936 - acc: 0.7813
Epoch 11/3

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.5min remaining:    0.0s


Epoch 1/30
3840/3840 [==============================] - 35s 9ms/step - loss: 0.6670 - acc: 0.6894
Epoch 2/30
3840/3840 [==============================] - 1s 219us/step - loss: 0.5845 - acc: 0.7497
Epoch 3/30
3840/3840 [==============================] - 1s 234us/step - loss: 0.5554 - acc: 0.7593
Epoch 4/30
3840/3840 [==============================] - 1s 218us/step - loss: 0.5393 - acc: 0.7655
Epoch 5/30
3840/3840 [==============================] - 1s 226us/step - loss: 0.5211 - acc: 0.7710
Epoch 6/30
3840/3840 [==============================] - 1s 220us/step - loss: 0.5028 - acc: 0.7742
Epoch 7/30
3840/3840 [==============================] - 1s 223us/step - loss: 0.4995 - acc: 0.7834
Epoch 8/30
3840/3840 [==============================] - 1s 236us/step - loss: 0.4818 - acc: 0.7778
Epoch 9/30
3840/3840 [==============================] - 1s 219us/step - loss: 0.4749 - acc: 0.7864
Epoch 10/30
3840/3840 [==============================] - 1s 236us/step - loss: 0.4612 - acc: 0.7959
Epoch 11/3

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  6.3min finished


In [21]:
# View correct results
# summarize results
means = [np.mean(x) for x in results]
stds = [np.std(x) for x in results]
params = list(itertools.product(batch_size_l, epochs))
print("Best: %f using %s" % (np.max(means), params[np.argmax(means)]))
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# Save into a variable
batchepoch2_results = results

# Best: 0.695380 using (35, 25)
# 0.692537 (0.022636) with: (25, 17)
# 0.685639 (0.018829) with: (25, 20)
# 0.682780 (0.025608) with: (25, 25)
# 0.684346 (0.017185) with: (25, 30)
# 0.691154 (0.014746) with: (32, 17)
# 0.688317 (0.008964) with: (32, 20)
# 0.678900 (0.006081) with: (32, 25)
# 0.689969 (0.019698) with: (32, 30)
# 0.673499 (0.013015) with: (35, 17)
# 0.685111 (0.019830) with: (35, 20)
# 0.695380 (0.011148) with: (35, 25)
# 0.681501 (0.022806) with: (35, 30)
# 0.674983 (0.012844) with: (40, 17)
# 0.682939 (0.014346) with: (40, 20)
# 0.683199 (0.012419) with: (40, 25)
# 0.686257 (0.026148) with: (40, 30)
# 0.688066 (0.006543) with: (50, 17)
# 0.683393 (0.011736) with: (50, 20)
# 0.679720 (0.006632) with: (50, 25)
# 0.683576 (0.021029) with: (50, 30)
# 0.673349 (0.011632) with: (60, 17)
# 0.681240 (0.025178) with: (60, 20)
# 0.682025 (0.024088) with: (60, 25)
# 0.683618 (0.008155) with: (60, 30)
# 0.671850 (0.019080) with: (70, 17)
# 0.684717 (0.025766) with: (70, 20)
# 0.687561 (0.023567) with: (70, 25)
# 0.687506 (0.017302) with: (70, 30)
# 0.676964 (0.013289) with: (80, 17)
# 0.676168 (0.016125) with: (80, 20)
# 0.684666 (0.025443) with: (80, 25)
# 0.686619 (0.010823) with: (80, 30)
# 0.666451 (0.020302) with: (90, 17)
# 0.679666 (0.015284) with: (90, 20)
# 0.680232 (0.026340) with: (90, 25)
# 0.673651 (0.027212) with: (90, 30)

Best: 0.695380 using (35, 25)
0.692537 (0.022636) with: (25, 17)
0.685639 (0.018829) with: (25, 20)
0.682780 (0.025608) with: (25, 25)
0.684346 (0.017185) with: (25, 30)
0.691154 (0.014746) with: (32, 17)
0.688317 (0.008964) with: (32, 20)
0.678900 (0.006081) with: (32, 25)
0.689969 (0.019698) with: (32, 30)
0.673499 (0.013015) with: (35, 17)
0.685111 (0.019830) with: (35, 20)
0.695380 (0.011148) with: (35, 25)
0.681501 (0.022806) with: (35, 30)
0.674983 (0.012844) with: (40, 17)
0.682939 (0.014346) with: (40, 20)
0.683199 (0.012419) with: (40, 25)
0.686257 (0.026148) with: (40, 30)
0.688066 (0.006543) with: (50, 17)
0.683393 (0.011736) with: (50, 20)
0.679720 (0.006632) with: (50, 25)
0.683576 (0.021029) with: (50, 30)
0.673349 (0.011632) with: (60, 17)
0.681240 (0.025178) with: (60, 20)
0.682025 (0.024088) with: (60, 25)
0.683618 (0.008155) with: (60, 30)
0.671850 (0.019080) with: (70, 17)
0.684717 (0.025766) with: (70, 20)
0.687561 (0.023567) with: (70, 25)
0.687506 (0.017302) with:

# NN2 results: CV - 0.6837588203178561, public - 0.6993
# NN1 results: CV ~ 0.696, public - 0.693 

# Try the Ensemble of SVM and NN

In [44]:
# --- NN Model --- #
# dropout in hidden layers with weight constraint
def create_model():
    # create model
    neurons = 30
    dropout_rate = 0.7
    weight_constraint = 4
    model = Sequential()
    model.add(Dense(neurons, input_dim=1000, activation='relu', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, activation='relu', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate*0.4))
    model.add(Dense(int(neurons/2), activation='relu', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate*0.2))
    model.add(Dense(3, activation='sigmoid'))
    
    # Change output
    def class_output_shape(input_shape):
        shape = list(input_shape)
        assert len(shape) == 2  # only valid for 2D tensors
        shape[-1] *= 1
        print(shape)
        return tuple(shape)
    def fn(y):
        return np.argmax(y, axis=1)
    
    model.add(Lambda(fn, output_shape=class_output_shape), output_shape=class_output_shape)
    model.add(Dense(1))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def antirectifier_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 2  # only valid for 2D tensors
    shape[-1] *= 2
    return tuple(shape)

# One hot encode data
y_enc = np.zeros((y.shape[0], 3))
y_enc[np.arange(y.shape[0]), y] = 1

# Build the estimator
mlp = KerasClassifier(build_fn=create_model, epochs=25, batch_size=35, verbose=1)
kfold = KFold(n_splits=5, shuffle=True)
cw = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y), y)
class_weight = {0: cw[0], 1: cw[1], 2: cw[2]}

def score(estimator, X_, y_):
    pred = estimator.predict(X_)
    if y_.shape[1] < 2:
        y_normal = y_
    else:
        y_normal = np.argmax(y_, axis=1)
    BMAC = balanced_accuracy_score(y_normal, pred)
    return BMAC

# # --- SVM model --- #
# orig_svm = svm.SVC(C=0.5, class_weight='balanced', degree=1, gamma='auto', kernel='rbf')

# # --- Ensemble --- #
# eclf = VotingClassifier(estimators=[('svm', orig_svm), ('mlp', mlp)], voting='hard', 
#                         fit_params={'mlp__class_weight': class_weight})

# steps = [("scaler", preprocessing.StandardScaler()), ("classifier", eclf)]
# pipeline = Pipeline(steps = steps)

# score = cross_val_score(pipeline, X, y, cv=kf, scoring=make_scorer(balanced_accuracy_score), verbose=2)
# print("Score:", score)

result = cross_val_score(pipeline, X, y, cv=kfold, verbose=3, scoring=make_scorer(balanced_accuracy_score), 
                          fit_params={'mlp__class_weight': class_weight})


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................


ValueError: Error when checking target: expected dense_1344 to have shape (3,) but got array with shape (1,)

# Sklearn NN grid search (not working)

In [ ]:
steps = [("scaler", preprocessing.StandardScaler()), ("classifier", SMOTEClassifier(SMOTE(), MLPClassifier()))]
pipeline = Pipeline(steps = steps)

parameters = {"classifier__solver": ['lbfgs'],
              "classifier__alpha": [1e-5, 1e-3, 0.1, 0.3, 0.5],
              "classifier__hidden_layer_sizes": [(30, 30), (80, 80), (150, 150)],
              "classifier__activation": ['relu', 'logistic'],
              "classifier__learning_rate": ["invscaling", "constant"], 
              "classifier__learning_rate_init": [0.0005, 0.001, 0.005, 0.01], 
              "classifier__beta_1": [0.8, 0.9, 0.999], 
              "classifier__beta_2": [0.8, 0.9, 0.999], 
              "classifier__early_stopping": [True], 
             }

grid = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'balanced_accuracy', verbose=1)

grid.fit(X, y)

# SKlearn NN and Ensemble

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True)

mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,
                       hidden_layer_sizes=(150,150), random_state=1,
                        activation='relu', learning_rate='invscaling', beta_1=0.9, 
                          beta_2=0.999, learning_rate_init=0.001, early_stopping=True,
                          momentum=0.9, shuffle=True, epsilon=1e-08, 

                      )
orig_svm = svm.SVC(C=0.5, class_weight='balanced', degree=1, gamma='auto', kernel='rbf')
eclf = VotingClassifier(estimators=[('svm', orig_svm), ('mlp', mlp)], voting='hard')

steps = [("scaler", preprocessing.StandardScaler()), ("classifier", eclf)]
pipeline = Pipeline(steps = steps)

score = cross_val_score(pipeline, X, y, cv=kf, scoring=make_scorer(balanced_accuracy_score), verbose=2)
print("Score:", score)

# Ensemble score: Score: [0.66481481 0.64861111 0.67824074 0.6462963  0.65231481]
# SVM score: Score: [0.69907407 0.70740741 0.69583333 0.67962963 0.70416667]
# Result: Ensemble MLP (no SMOTE) + SVM is bad, and no SMOTE 